In [1]:
import numpy as np
import parsl
from parsl import python_app
from parsl.config import Config
from parsl.executors.threads import ThreadPoolExecutor
import pandas as pd
import os
import re
import pandas as pd
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity
import time 
import json

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = Config(executors=[ThreadPoolExecutor()])
parsl.load(config)

In [6]:
def extractTag(text):
  matches = re.findall(r'\"tags\": \[(.*?)\]', text, re.DOTALL)
  matches = re.findall(r'\"(.*?)\"', matches[0].strip(','))
  return matches

In [3]:
#Constroi a lista com os caminhos dos workflows
os.chdir(os.path.join(os.pardir,os.pardir, 'workflows_galaxy'))
nomes_arquivos = []
diretorio = os.getcwd()

for item in os.listdir(diretorio):
    caminho_completo = os.path.join(diretorio, item)
    if os.path.isfile(caminho_completo):
        nomes_arquivos.append(item)

#nomes_arquivos = nomes_arquivos[0:10]

print(len(nomes_arquivos))

1014


In [9]:
dic_workflows = {}
@python_app
def processar_arquivo(arquivo):
    # Carregar o arquivo JSON
    with open(arquivo, 'r') as file:
        fileName = file.name
        dados = file.read()
        matches_tag = extractTag(dados)
        dic_workflows[fileName] = dados

        combined_results = {
        'Tags': matches_tag if matches_tag else [],  # Lidar com o cenário de não ter tags
        'Content': dados
        }

        dic_workflows[fileName] = combined_results

        return dic_workflows[fileName]


# Inicia o processamento dos arquivos em paralelo
futures = [processar_arquivo(arquivo) for arquivo in nomes_arquivos]

resultados = [future.result() for future in futures]

In [13]:
df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()
df

,index,Tags,Content
0,0c86c39dcd9e08c6.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
1,692a2b0bb818336d.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
2,3680984663c813e1.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
3,49f8b32c3206f76c.json,"[variant, snps, human]","{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
4,25adc55d2a26e34b.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
...,...,...,...
1009,d5cba8a5ba6880fd.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
1010,e0da87cadb1e6d5f.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
1011,fbf75fbb72b488bd.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."
1012,0d8eb75d28fa7df2.json,[],"{\n ""a_galaxy_workflow"": ""true"",\n ""anno..."


In [14]:
# Função para converter texto em embedding
def text_to_embedding(text, tokenizer, device, model):
    encoded_input = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    encoded_input = {key: value.to(device) for key, value in encoded_input.items()}  # Mover tensores para o dispositivo
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()  # Mover o resultado de volta para a CPU e converter para numpy

In [15]:
df = pd.DataFrame.from_dict(dic_workflows, orient='index').reset_index()

# Definir a lista de valores de tops a serem calculados
top_x_list = list(range(1,2))  # Você pode alterar ou adicionar mais valores a esta lista

# DataFrame para armazenar os tempos de execução
execution_times = []

# Calcular os top_x para cada valor na lista top_x_list
# Calcular os top_x para cada valor na lista top_x_list
for top_x in top_x_list:
    # Medir o tempo de execução
    start_time = time.time()

    # Carregar o modelo e o tokenizer
    tokenizer = BertTokenizer.from_pretrained('google-bert/bert-large-uncased')
    model = BertModel.from_pretrained('google-bert/bert-large-uncased')

    # Definir o dispositivo (GPU ou CPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)

    # Supondo que você já tem um DataFrame df com uma coluna 'text'
    texts = df['Content'].tolist()

    # Converter todos os textos para embeddings
    embeddings = [text_to_embedding(text, tokenizer, device, model) for text in texts]

    # Converter a lista de embeddings em um array 2D
    embeddings_array = np.vstack(embeddings)

    # Calcular a matriz de similaridade
    similarity_matrix = cosine_similarity(embeddings_array)
    
    # Identificar os índices dos textos mais similares para cada texto
    similar_indices = similarity_matrix.argsort(axis=1)[:, :-top_x-2:-1]  # Selecionar os top_x mais similares excluindo o próprio texto

    # Remover o índice do próprio texto
    corrected_similar_indices = []
    corrected_similar_tags = []
    for idx, indices in enumerate(similar_indices):
        filtered_indices = [index for index in indices if index != idx][:top_x]  # Exclui o próprio e pega os top_x mais similares
        filtered_tags = [df.iloc[index]['Tags'] for index in filtered_indices]  # Obter as tags dos textos mais similares
        corrected_similar_indices.append(filtered_indices)
        corrected_similar_tags.append(filtered_tags)

    # Criar coluna no DataFrame para os índices dos textos mais similares
    df[f'top{top_x}_description'] = corrected_similar_indices

    # Criar coluna no DataFrame para as tags dos textos mais similares
    df[f'top{top_x}_tags'] = corrected_similar_tags

    # Calcular a média das similaridades dos textos mais similares para cada texto
    mean_similarities = []
    for idx, indices in enumerate(corrected_similar_indices):
        similarities = [similarity_matrix[idx, i] for i in indices]
        mean_similarity = np.mean(similarities)
        mean_similarities.append(mean_similarity)

    df[f'mean_similarity_top{top_x}'] = mean_similarities

    # Calcular o tempo de execução
    execution_time = time.time() - start_time

    # Adicionar o tempo de execução ao DataFrame de tempos de execução
    execution_times.append({'top_x': top_x, 'execution_time': execution_time})

# Exibir o DataFrame com os tempos de execução
execution_times_df = pd.DataFrame(execution_times)
print(execution_times_df)

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


   top_x  execution_time
0      1      187.725738


In [8]:
# Carregar o modelo e o tokenizer
tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased')

texts = df['text'].tolist()

# Função para converter texto em embedding
def text_to_embedding(text):
    encoded_input = tokenizer(text, return_tensors='pt', max_length=512, padding=True, truncation=True)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Pegar a média dos embeddings de todos os tokens para representar o texto
    return model_output.last_hidden_state.mean(dim=1).squeeze().numpy()

# Converter todos os textos para embeddings
embeddings = [text_to_embedding(text) for text in texts]

# Converter a lista de embeddings em um array 2D
embeddings_array = np.vstack(embeddings)

# Calcular a matriz de similaridade
similarity_matrix = cosine_similarity(embeddings_array)

# Identificar os índices dos três textos mais similares para cada texto
similar_indices = similarity_matrix.argsort(axis=1)[:, :-4:-1]  # -4 porque os três mais similares excluem o próprio texto

# Criar coluna no DataFrame para os índices dos textos mais similares
df['top3_descrition_scibert'] = similar_indices.tolist()

# Criar coluna no DataFrame para os textos mais similares
#df['top3_similar_text_descrition_scibert'] = [[df.loc[idx, 'text'] for idx in indices] for indices in similar_indices]

/home/lyncoln/Git/similaridade_workflow/venv_linux/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
# os.chdir(os.pardir)
df.to_csv("top3_json.csv")

In [14]:
os.getcwd()

'/home/lyncoln/Git'